In [1]:
!pip install transformers datasets bitsandbytes accelerate trl torch tensorboard tqdm
!pip install -U bitsandbytes
!pip install peft==0.13.0

In [2]:
from datasets import load_dataset, DatasetDict, Dataset,load_from_disk

In [3]:
import json
from pathlib import Path
from huggingface_hub import hf_hub_download
from datasets import Dataset, DatasetDict

def load_prm800k_direct(split="all", cache_dir=None, limit=None):
    """
    Load PRM800K dataset directly from JSONL files

    Args:
        split: "train", "test", or "all" (default: "all")
        cache_dir: Directory to cache downloaded files
        limit: Limit number of examples per split (for testing)

    Returns:
        Dataset or DatasetDict depending on split parameter
    """

    if cache_dir is None:
        cache_dir = Path.home() / ".cache" / "prm800k_jsonl"
    else:
        cache_dir = Path(cache_dir)

    cache_dir.mkdir(parents=True, exist_ok=True)

    dataset_id = "tasksource/PRM800K"

    # Map split names to files
    split_files = {
        "train": ["phase1_train.jsonl", "phase2_train.jsonl"],
        "test": ["phase1_test.jsonl", "phase2_test.jsonl"],
        "all": ["phase1_train.jsonl", "phase2_train.jsonl", "phase1_test.jsonl", "phase2_test.jsonl"]
    }

    if split not in split_files:
        raise ValueError(f"split must be 'train', 'test', or 'all'. Got: {split}")

    files_to_load = split_files[split]

    print(f"📥 Loading PRM800K ({split} split)...")

    # Download and load files
    datasets_dict = {}

    for filename in files_to_load:
        # Determine the split name for this file
        if "train" in filename:
            split_name = "train"
        else:
            split_name = "test"

        if split_name not in datasets_dict:
            datasets_dict[split_name] = []

        print(f"   📄 Downloading {filename}...")
        try:
            filepath = hf_hub_download(
                repo_id=dataset_id,
                filename=filename,
                repo_type="dataset",
                cache_dir=str(cache_dir),
                force_download=False
            )

            print(f"   📖 Loading {filename}...")
            with open(filepath, 'r', encoding='utf-8') as f:
                count = 0
                for line in f:
                    if line.strip():
                        try:
                            example = json.loads(line)
                            datasets_dict[split_name].append(example)
                            count += 1
                            if limit and count >= limit:
                                break
                        except json.JSONDecodeError:
                            pass

                print(f"      ✅ Loaded {count} examples")

        except Exception as e:
            print(f"      ⚠️  Error loading {filename}: {e}")

    # Convert to HuggingFace Dataset objects
    result = {}
    for split_name, examples in datasets_dict.items():
        if examples:
            result[split_name] = Dataset.from_dict({
                key: [ex.get(key) for ex in examples]
                for key in examples[0].keys()
            })

    print(f"\n✅ Dataset loaded!")

    # Return format matching load_dataset behavior
    if split == "all":
        return DatasetDict(result)
    else:
        return result.get(split)

In [4]:
# For loading the actual test split
test_data = load_prm800k_direct(split="test")
train_data = load_prm800k_direct(split="train")
train_data_overview = load_prm800k_direct(split="train", limit=2)

📥 Loading PRM800K (test split)...
   📄 Downloading phase1_test.jsonl...
   📖 Loading phase1_test.jsonl...
      ✅ Loaded 106 examples
   📄 Downloading phase2_test.jsonl...
   📖 Loading phase2_test.jsonl...
      ✅ Loaded 2762 examples

✅ Dataset loaded!
📥 Loading PRM800K (train split)...
   📄 Downloading phase1_train.jsonl...
   📖 Loading phase1_train.jsonl...
      ✅ Loaded 949 examples
   📄 Downloading phase2_train.jsonl...
   📖 Loading phase2_train.jsonl...
      ✅ Loaded 97782 examples

✅ Dataset loaded!
📥 Loading PRM800K (train split)...
   📄 Downloading phase1_train.jsonl...
   📖 Loading phase1_train.jsonl...
      ✅ Loaded 2 examples
   📄 Downloading phase2_train.jsonl...
   📖 Loading phase2_train.jsonl...
      ✅ Loaded 2 examples

✅ Dataset loaded!


In [5]:
print(json.dumps(train_data_overview[0], indent=2))

{
  "labeler": "e90a38f3-3135-4465-87af-3e6322e3d772",
  "timestamp": "2022-07-13T18:55:54.496450",
  "generation": null,
  "is_quality_control_question": false,
  "is_initial_screening_question": false,
  "question": {
    "ground_truth_answer": "468",
    "ground_truth_solution": null,
    "pre_generated_answer": null,
    "pre_generated_steps": null,
    "pre_generated_verifier_score": null,
    "problem": "How many seconds are in 7.8 minutes?"
  },
  "label": {
    "finish_reason": "solution",
    "steps": [
      {
        "chosen_completion": 0,
        "completions": [
          {
            "flagged": false,
            "rating": 1,
            "text": "7.8 minutes is the same as 7 minutes and 0.8 minutes."
          }
        ],
        "human_completion": null
      },
      {
        "chosen_completion": 0,
        "completions": [
          {
            "flagged": false,
            "rating": 1,
            "text": "Right, and since there are 60 seconds in a minute, then 

In [6]:
print(json.dumps(train_data_overview[1], indent=2))

{
  "labeler": "e90a38f3-3135-4465-87af-3e6322e3d772",
  "timestamp": "2022-07-17T16:56:51.323252",
  "generation": null,
  "is_quality_control_question": false,
  "is_initial_screening_question": false,
  "question": {
    "ground_truth_answer": "12",
    "ground_truth_solution": null,
    "pre_generated_answer": null,
    "pre_generated_steps": null,
    "pre_generated_verifier_score": null,
    "problem": "How many positive two-digit integers leave a remainder of 2 when divided by 8?"
  },
  "label": {
    "finish_reason": "solution",
    "steps": [
      {
        "chosen_completion": 6,
        "completions": [
          {
            "flagged": false,
            "rating": 0,
            "text": "Let's call our two-digit integers x."
          },
          {
            "flagged": false,
            "rating": 0,
            "text": "Let's first think about the remainders when we divide by 8."
          },
          {
            "flagged": false,
            "rating": 1,
        

In [7]:
import json
import random
from typing import List, Dict
from datasets import Dataset
from tqdm import tqdm

In [8]:
def extract_step_data(item: Dict) -> List[Dict]:
    """
    Extract step-by-step reasoning data with ratings.
    Output format: problem + partial solution -> next step rating
    """
    examples = []

    problem = item['question']['problem']
    steps = item['label']['steps']
    ground_truth = item['question'].get('ground_truth_answer', '')

    # Build conversation incrementally
    conversation_history = []

    for step_idx, step in enumerate(steps):
        if step['chosen_completion'] is None:
            continue

        chosen_idx = step['chosen_completion']
        chosen_step = step['completions'][chosen_idx]

        # Create the prompt with conversation history
        if len(conversation_history) == 0:
            prompt = f"Problem: {problem}\n\nSolution steps so far:\n"
        else:
            prompt = f"Problem: {problem}\n\nSolution steps so far:\n"
            for i, prev_step in enumerate(conversation_history, 1):
                prompt += f"Step {i}: {prev_step}\n"

        prompt += f"\nStep {len(conversation_history) + 1}: {chosen_step['text']}"

        # Rating: -1 (bad), 0 (neutral), 1 (good)
        rating = chosen_step['rating']
        rating_text = {
            -1: "incorrect or unhelpful",
            0: "neutral or partially correct",
            1: "correct and helpful"
        }.get(rating, "unknown")

        # Create label with rating and optional comment
        label = f"Rating: {rating} ({rating_text})"
        if chosen_step.get('flagged', False):
            label += " [FLAGGED]"

        examples.append({
            'text': prompt,
            'label': label,
            'rating': rating,
            'step_number': len(conversation_history) + 1,
            'total_steps': len(steps),
            'problem': problem,
            'ground_truth': ground_truth
        })

        # Add chosen step to history
        conversation_history.append(chosen_step['text'])

        # Optionally include negative examples (unchosen completions)
        # This helps the model learn what NOT to do
        for comp_idx, completion in enumerate(step['completions']):
            if comp_idx != chosen_idx and completion.get('rating') is not None and completion['rating'] <= 0:
                negative_prompt = prompt.replace(
                    chosen_step['text'],
                    completion['text']
                )

                neg_rating_text = {
                    -1: "incorrect or unhelpful",
                    0: "neutral or partially correct",
                    1: "correct and helpful"
                }.get(completion['rating'], "unknown")

                negative_label = f"Rating: {completion['rating']} ({neg_rating_text})"
                if completion.get('flagged', False):
                    negative_label += " [FLAGGED]"

                examples.append({
                    'text': negative_prompt,
                    'label': negative_label,
                    'rating': completion['rating'],
                    'step_number': len(conversation_history),
                    'total_steps': len(steps),
                    'problem': problem,
                    'ground_truth': ground_truth,
                    'is_negative': True
                })

    return examples

In [9]:
def format_for_training(example: Dict, include_label: bool = True) -> str:
    """
    Format example in a chat-like format suitable for Phi-2.
    """
    if include_label:
        formatted = f"{example['text']}\n\nEvaluation: {example['label']}"
    else:
        formatted = example['text']

    return formatted

In [10]:
def prepare_dataset(data: List[Dict], output_file: str = None,
                   include_negatives: bool = True,
                   max_samples: int = None):
    """
    Prepare the complete dataset for training.
    """
    all_examples = []

    print("Processing PRM800K data...")
    for item in tqdm(data):
        examples = extract_step_data(item)

        if not include_negatives:
            # Filter out negative examples
            examples = [ex for ex in examples if not ex.get('is_negative', False)]

        all_examples.extend(examples)

    print(f"Total examples extracted: {len(all_examples)}")

    # Shuffle
    random.shuffle(all_examples)

    # Limit samples if specified
    if max_samples:
        all_examples = all_examples[:max_samples]
        print(f"Limited to {max_samples} samples")

    # Format for training
    formatted_data = []
    for ex in all_examples:
        formatted_data.append({
            'input_text': format_for_training(ex, include_label=False),
            'output_text': ex['label'],
            'full_text': format_for_training(ex, include_label=True),
            'rating': ex['rating'],
            'step_number': ex['step_number'],
            'total_steps': ex['total_steps']
        })

    # Save if output file specified
    if output_file:
        with open(output_file, 'w') as f:
            json.dump(formatted_data, f, indent=2)
        print(f"Saved to {output_file}")

    return formatted_data

In [11]:
def create_hf_dataset(formatted_data: List[Dict]):
    """
    Convert to HuggingFace Dataset format.
    """
    dataset_dict = {
        'input_text': [ex['input_text'] for ex in formatted_data],
        'output_text': [ex['output_text'] for ex in formatted_data],
        'full_text': [ex['full_text'] for ex in formatted_data],
        'rating': [ex['rating'] for ex in formatted_data],
    }

    return Dataset.from_dict(dataset_dict)

In [12]:
# Prepare datasets
print("\n=== Preparing Training Data ===")
train_formatted = prepare_dataset(
    train_data,
    output_file='train_formatted.json',
    include_negatives=True,
    max_samples=None  # Use all data
)

print("\n=== Preparing Test Data ===")
test_formatted = prepare_dataset(
    test_data,
    output_file='test_formatted.json',
    include_negatives=True,
    max_samples=None
)

# Create HuggingFace datasets
train_dataset = create_hf_dataset(train_formatted)
test_dataset = create_hf_dataset(test_formatted)

# Print statistics
print("\n=== Dataset Statistics ===")
print(f"Training examples: {len(train_dataset)}")
print(f"Test examples: {len(test_dataset)}")

print("\nRating distribution (train):")
ratings = train_dataset['rating']
for r in [-1, 0, 1]:
    count = sum(1 for rating in ratings if rating == r)
    print(f"  Rating {r}: {count} ({count/len(ratings)*100:.1f}%)")

print("\n=== Sample Example ===")
print(train_formatted[0]['full_text'][:500] + "...")


=== Preparing Training Data ===
Processing PRM800K data...


100%|██████████| 98731/98731 [00:26<00:00, 3785.46it/s]


Total examples extracted: 641863
Saved to train_formatted.json

=== Preparing Test Data ===
Processing PRM800K data...


100%|██████████| 2868/2868 [00:00<00:00, 3501.25it/s]


Total examples extracted: 20185
Saved to test_formatted.json

=== Dataset Statistics ===
Training examples: 641863
Test examples: 20185

Rating distribution (train):
  Rating -1: 9979 (1.6%)
  Rating 0: 56965 (8.9%)
  Rating 1: 541039 (84.3%)

=== Sample Example ===
Problem: In a solar system of $n$ planets, Zorn the World Conqueror can invade $m$ planets at a time, but once there are less than $m$ free worlds left, he stops. If he invades $13$ at a time then there are $6$ left, and if he invades $14$ at a time then there are $5$ left. If this solar system has more than $100$ planets, what is the smallest number of planets it could have?

Solution steps so far:
Step 1: I need to find the smallest $n$ that satisfies two congruences: $n \equiv 6 \pmod{13}$ an...


In [13]:
# Load the formatted data
with open('train_formatted.json', 'r') as f:
    train_formatted = json.load(f)

with open('test_formatted.json', 'r') as f:
    test_formatted = json.load(f)

print(f"Training examples: {len(train_formatted)}")
print(f"Test examples: {len(test_formatted)}")

# Create HuggingFace datasets
from datasets import Dataset

train_dataset = Dataset.from_dict({
    'text': [ex['full_text'] for ex in train_formatted]
})

test_dataset = Dataset.from_dict({
    'text': [ex['full_text'] for ex in test_formatted]
})

print("✅ Datasets ready for training!")

Training examples: 641863
Test examples: 20185
✅ Datasets ready for training!


# Initialization Tests

In [14]:
# Check installations
import sys
print(f"Python version: {sys.version}")

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda if torch.cuda.is_available() else 'N/A'}")

import transformers
print(f"Transformers version: {transformers.__version__}")

import peft
print(f"PEFT version: {peft.__version__}")

import bitsandbytes
print(f"BitsAndBytes version: {bitsandbytes.__version__}")

Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
Transformers version: 4.57.1


2025-11-06 01:09:51.717727: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762391391.877722     167 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762391391.923821     167 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


PEFT version: 0.13.0
BitsAndBytes version: 0.48.2


In [15]:
# from transformers import AutoModelForCausalLM

# # Try loading without quantization first
# model = AutoModelForCausalLM.from_pretrained(
#     "microsoft/phi-2",
#     device_map="cpu",  # or "auto"
#     trust_remote_code=True,
#     torch_dtype=torch.float32,
# )

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [16]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model = prepare_model_for_kbit_training(model)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✅ Model loaded!")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

✅ Model loaded!


In [17]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "dense", "fc1", "fc2"],
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 94,371,840 || all params: 2,874,055,680 || trainable%: 3.2836


In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Training arguments
training_args = TrainingArguments(
    output_dir="./phi2-prm-qlora",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    optim="paged_adamw_32bit",
    weight_decay=0.001,
    logging_steps=10,
    save_steps=500,
    eval_steps=500,
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    bf16=True,
    gradient_checkpointing=True,
    report_to="tensorboard",
    save_total_limit=2,
)

# Formatting function
def formatting_func(example):
    return example['text']

# Create trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    formatting_func=formatting_func,
)

# Start training!
print("🚀 Starting training...")
trainer.train()

print("✅ Training complete!")

Applying formatting function to train dataset:   0%|          | 0/641863 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/641863 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/641863 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2188 > 2048). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/641863 [00:00<?, ? examples/s]

...

Adding EOS to eval dataset:   0%|          | 0/20185 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/20185 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/20185 [00:00<?, ? examples/s]

🚀 Starting training...


Step,Training Loss,Validation Loss


In [ ]:
# Save final model
trainer.save_model("./phi2-prm-final")
tokenizer.save_pretrained("./phi2-prm-final")

print("💾 Model saved!")